\tableofcontents

\pagebreak

# Consideraciones Generales

Este informe documenta el desarrollo de una aplicación de línea de comandos para adquisición y gestión de datos desde un microcontrolador, utilizando C++17 y una arquitectura simple orientada a objetos. La herramienta fue probada con hardware real y contempla lectura/escritura de archivos, conversión entre formatos y comunicación serie.

## Alcance

Se implementa una aplicación de consola orientada a objetos que integra tres ejes: comunicación (transporte serie), serialización/deserialización (CSV/JSON/XML) y persistencia en archivos locales. El objetivo es leer o emitir mensajes estructurados y registrarlos en disco con control básico de errores.

## Hipótesis y supuestos de diseño

- Transporte. El puerto serie opera a un baudrate fijo (p.ej. 115200 8N1), lectura línea a línea con terminador `\n`, buffer suficiente para un mensaje completo, y timeout finito configurable. Se asume un solo productor de datos y encolado FIFO.

- Formato de datos. Cada mensaje llega completo por línea. Si el modo elegido es:
    - CSV: campos separados por comas/punto y coma, sin saltos de línea embebidos.
    - JSON: un objeto por línea, UTF-8 válido.
    - XML: elemento raíz por línea o bloque bien formado recuperable por acumulación.

- Dominio. Types.h define tipos como Mode (lectura/escritura/append), IOFormat (CSV/JSON/XML) y Message (código/ payload/metadata). El valor por defecto de code se interpreta como “OK” salvo que la deserialización indique lo contrario.

- Errores. Fallas de E/S o parseo se gestionan con excepciones y/o códigos de estado encapsulados; la app registra incidentes y continúa según política de tolerancia.

- Persistencia. La escritura es a FS local usando flujos C++ estándar con apertura segura y cierre garantizado. No se usa base de datos.

## Principios OO aplicados

- __Encapsulamiento__: detalles de E/S serie y de formato se ocultan detrás de ITransport e ISerializer. La app solo “pide servicios”.

- __Modularidad y compilación separada__: headers/definitions por componente, enlazado posterior.

- __Bajo acoplamiento, alta cohesión__: cada clase tiene una única responsabilidad clara (GRASP “experto” y “creador”).

- __Relaciones UML__: asociaciones dirigidas App→Transport/Serializer y composición App→Repo de archivos.



# Esquema general de la solución

## Diseño de la solución
Se adoptó una arquitectura simple orientada a objetos, respetando el diagrama de clases provisto en la consigna. El diagrama actualizado se ilustra a continuación:


- **App** coordina la ejecución general y delega en `CLI` la interpretación de argumentos.
- **CLI** encapsula la lectura de los parámetros y expone *getters* tipados (`Mode`, `IOFormat`).
- **File** maneja la persistencia en CSV, mantiene los metadatos (`FileInfo`) y ofrece conversores a JSON/XML.
- **SerialPort** abstrae la comunicación por puerto serie usando `termios` para trabajar en modo crudo a 115200 bauds.
- **Types** concentra tipos comunes (`AppError`, `Mode`, `IOFormat`) y las funciones utilitarias `parseLine` y `tryParseHeaderTag`.

![alt text](DiagramUmbrello.png)


# Interfaces de usuario

La interfaz es puramente CLI. Los casos de uso principales son:

- Escritura (captura desde $\mu C$ a CSV)
```bash
./app -m w -i c -n 20 -s /dev/ttyACM0 -d logs -f sensor
```

Lectura en distintos formatos

- CSV:       
    
```bash
./app -m r -o c -d logs -f sensor
```
 
- JSON:


```bash
./app -m r -o j -d logs -f sensor
```

- XML:

```bash
./app -m r -o x -d logs -f sensor
```

Luego, tenemos las siguientes capturas de pantalla:

![alt text](Imagenes/captura1.png)

![alt text](Imagenes/captura2.png)


Cabe resaltar que usar el comando `cat` me muestra los datos del archivo. Esto quiere decir que la **persistencia** se realiza en formato `.csv`.


![alt text](Imagenes/captura_csv.png)

![alt text](Imagenes/captura_json.png)

![alt text](Imagenes/captura_xml.png)


# Recursos adicionales

No se hace uso de ningun componente de software no estandar del lenguaje ni de la plataforma para codificar el programa. A continuación, se listan las librerias estandar de C++ y de Linux que se utilizan en los diferentes archivos `.h` y `.cpp`.

```C++
#include <string>
#include <vector>

#include <fstream>
#include <stdexcept>
#include <termios.h>
#include <chrono>
#include <ctime>
#include <sstream>
#include <iomanip>
#include <iostream>
#include <cstdlib>
#include <sys/stat.h>
#include <unistd.h>
#include <pwd.h>
#include <filesystem>
#include <system_error>
#include <algorithm>

```

## ¿Para qué sirven los `#include`?

- `<string>`: `std::string`, cadenas decentes, no arrays C.

- `<vector>`: `std::vector<T>`, arrays dinámicos.

- `<fstream>`: `std::ifstream/ofstream/fstream` para leer/escribir archivos.

- `<stdexcept>`: excepciones estándar (`std::runtime_error`, `std::invalid_argument…`).

- `<termios.h>`: control POSIX de puertos/terminales. Configurar serial: baudrate, paridad, flags.

- `<chrono>`: tiempo moderno C++: steady_clock, milliseconds, sleep_for.

- `<ctime>`: tiempo estilo C: `time_t`, `std::localtime`, `std::strftime-like`.

- `<sstream>`: `std::stringstream` para parsear/armar strings como si fueran streams.

- `<iomanip>`: formateo de streams: `std::setw`, `std::setprecision`, `std::put_time`.

- `<iostream>`: `std::cout`, `std::cin`, `std::cerr`. El trío inevitable.

- `<cstdlib>`: utilidades varias: `std::getenv`, `std::system`, `std::strtol`, `std::rand`.

- `<sys/stat.h>`: stat, permisos y mkdir con modos; funciones del Sistema Operativo.

- `<unistd.h>`: POSIX: read, write, close, usleep, access, isatty.

- `<pwd.h>`: info de usuario: getpwuid para home, etc.

- `<filesystem>`: C++17 paths y archivos: `std::filesystem::path`, `exists`, `create_directories`.

- `<system_error>`: `std::error_code` y `std::system_error` para reportes de errores del Sistema Operativo.

- `<algorithm>`: `std::sort`, `std::find`, `std::transform`, `std::accumulate…` herramientas útiles.

# Manual de instrucciones de la aplicación

##  Uso (manual breve)

### Compilación

Desde el directorio del código (./Codigos):

```bash
make
```

### Captura (modo escritura)

Utilizandose como $\mu C$ un ESP32 de placa de desarrollo DOIT Devkit v1, se ve el directorio `/dev/ttyACMX` para hacer lectura de la comunicación serial entre el μC y la aplicación (equipo).

Se escribe en la carpeta donde se realizó el make:
```bash
./app -m w -i c -n 10 -s /dev/ttyACMO -d logs -f sensor
```

Donde:
• `-m w`: modo escritura
• `-i c|j|x`: formato de entrada esperado (CSV/JSON/XML “liviano”)
• `-n`: cantidad de lecturas
• `-s`: dispositivo serie
• `-d` y `-f`: directorio y nombre base del archivo

### Lectura/serialización (modo lectura)

```bash
./app -m r -o c -d logs -f sensor # CSV
./app -m r -o j -d logs -f sensor # JSON
./app -m r -o x -d logs -f sensor # XM
```

## Flujo en modo escritura
1. El operador define el formato esperado mediante `-i`.
2. `App` abre/crea el CSV en el directorio indicado y realiza, si es necesario, la cabecera `#h:`.
3. Para cada muestra se envía el byte de *handshake* (`c`, `j` o `x`) y se espera la respuesta del $\mu C$.
4. `parseLine` detecta el formato real, normaliza la estructura y devuelve un vector de campos.
5. `File::writeParsed` escapa los valores y los persiste siempre en CSV.
6. Se contabilizan las lecturas correctas y, al finalizar, se informa la tabla de metadatos.


## Flujo en modo lectura
1. `CLI` determina el formato deseado (`-o`).
2. `File` abre el CSV, actualiza los metadatos y expone `getInfoTable()`.
3. En función del formato elegido se reutiliza el CSV original (`toCSV()`), se arma un arreglo JSON (`toJSON()`) o un documento XML (`toXML()`).
4. Cuando no existen encabezados explícitos se generan identificadores genéricos (`c1`, `c2`, ...).


## Serialización y robustez
- Se configuran 115200 bauds, modo 8N1 sin control de flujo hardware.
- El `timeout` de lectura se fijó en 2000 ms para compensar jitter en la transmisión.
- Las líneas incompletas o vacías se descartan sin afectar el contador de lecturas exitosas.
- `File::countRowsOnDisk()` y `readAllRows()` ignoran comentarios y filas vacías para mantener una dimensión realista del dataset.


## Pruebas realizadas
- Compilación con `make` (usar desde `Codigos/`).
- Captura de 20 muestras desde el $\mu C$ con `./app -m w -i c -n 20 -s /dev/ttyACM0 -d logs -f sensor`.
- Visualización en los tres formatos disponibles: `./app -m r -o c`, `./app -m r -o j`, `./app -m r -o x`.
- Verificación manual de la consistencia del CSV resultante (`logs/sensor.csv`).


# Conclusiones

La aplicación cumple con los objetivos del TP: separa responsabilidades, estandariza formatos heterogéneos y simplifica la interacción con el $\mu C$ mediante un protocolo simple de handshakes. La arquitectura facilita extender nuevas fuentes/destinos de datos y admitir otros formatos. 


# Referencias consultadas

Documentación de la biblioteca estándar de C++ (containers, streams, <filesystem>).

Páginas de manual POSIX para termios, read/write/close, stat, getpwuid.

Apuntes de cátedra de POO (sección “Guía de Trabajos Prácticos”).




# Anexo

## Definición de clases

Se dejan los headers utilizados, los cuales dan una idea de cómo se hicieron las clases.


```C++
#ifndef TYPES_H
#define TYPES_H

#include <stdexcept>
#include <string>
#include <string>
#include <vector>


class AppError : public std::runtime_error {
public:
    explicit AppError(const std::string& msg) : std::runtime_error(msg) {}
};

enum class Mode { Read, Write };
enum class IOFormat { CSV, JSON, XML };

inline IOFormat charToFormat(char c) {
    switch (c) {
        case 'c': case 'C': return IOFormat::CSV;
        case 'j': case 'J': return IOFormat::JSON;
        case 'x': case 'X': return IOFormat::XML;
        default: throw std::invalid_argument("Formato inválido (use c|j|x).");
    }
}
inline const char* fmtName(IOFormat f) {
    switch (f) { case IOFormat::CSV: return "CSV"; case IOFormat::JSON: return "JSON"; default: return "XML"; }
}

// Convierte una línea recibida en formato c/j/x a un vector de strings.
// c => CSV; j => JSON plano tipo {"a":1,"b":2}; x => XML plano <row><a>1</a>...</row>
std::vector<std::string> parseLine(IOFormat inFmt, const std::string& line,
                                   std::vector<std::string>* headersOpt = nullptr);

// Encabezados: para CSV se asumen si vienen en la primera fila con '#h: a,b,c'
// Para JSON/XML se infieren del primer objeto si no se proveen.
bool tryParseHeaderTag(const std::string& line, std::vector<std::string>& headers);


#endif
```

```C++
#ifndef FILE_H
#define FILE_H

#include "Types.h"
#include <string>
#include <vector>
#include <fstream>

struct FileInfo {
    std::string name;
    std::string creationDateTime;
    std::string owner;
    size_t      dimension = 0; // líneas de datos
};

class File {
    bool      m_read  = false;
    bool      m_write = false;
    FileInfo  m_info;
    std::fstream m_fs;

public:
    File() = default;
    explicit File(const std::string& path);

    bool open(char mode); // 'r' o 'w' o 'a'
    void close();

    const FileInfo& info() const { return m_info; }
    std::string     getInfoTable() const;

    bool        exist() const;
    std::string getNombre() const;
    std::string getFecha() const;
    std::string getPropietario() const;
    size_t      getDimension() const;
    std::string getInfo() const;

    // Escritura
    void writeRaw(const std::string& line);                 // escribe tal cual (CSV ya formateado)
    void writeParsed(const std::vector<std::string>& fields); // une por coma

    // Lectura
    bool   getLine(std::string& out);
    std::string readAll(); // contenido CSV completo (sin cabecera especial)
    std::vector<std::vector<std::string>> readAllRows();

    // Presentación (a partir del CSV)
    std::string toCSV();                                   // igual al archivo
    std::string toJSON(const std::vector<std::string>& headers);
    std::string toXML (const std::vector<std::string>& headers);

    // Utilidad
    static std::vector<std::string> splitCSV(const std::string& line);
    void writeHeaderTag(const std::vector<std::string>& headers);
    
private:
    size_t m_rows = 0;  // filas de datos (excluye cabeceras #h)
    size_t countRowsOnDisk() const;
};

#endif
```

```C++
#ifndef SERIAL_H
#define SERIAL_H

#include <string>
#include <termios.h>

class SerialPort {
    int      fd_ = -1;
    termios  old_{};
public:
    // baud: B9600, B115200, etc.
    SerialPort(const std::string& dev, int baud);
    ~SerialPort();

    void writeByte(char c);
    void writeData(const void* data, size_t n);

    // Lee hasta '\n' o '\r' con timeout en ms. Devuelve true si obtuvo una línea.
    bool readLine(std::string& out, int timeout_ms);

    // >>> NUEVO: limpia el buffer de entrada (evita “o>”, restos, etc.)
    void flushIn();

    // no copiable
    SerialPort(const SerialPort&) = delete;
    SerialPort& operator=(const SerialPort&) = delete;
};


#endif
```

```C++
#ifndef CLI_H
#define CLI_H

#include "Types.h"
#include <string>

class CLI {
    Mode        m_mode = Mode::Read;
    IOFormat    m_inFmt  = IOFormat::CSV;   // para -m w
    IOFormat    m_outFmt = IOFormat::CSV;   // para -m r
    bool        m_useSerial = false;
    std::string m_serialDev;
    std::string m_baseDir  = ".";
    std::string m_fileName = "datos";
    bool        m_append   = false;
    int         m_readCount = 0;

public:
    void parse(int argc, char** argv);

    Mode       mode()       const { return m_mode; }
    IOFormat   inputFmt()   const { return m_inFmt; }
    IOFormat   outputFmt()  const { return m_outFmt; }
    bool       useSerial()  const { return m_useSerial; }
    std::string serialDev() const { return m_serialDev; }
    std::string baseDir()   const { return m_baseDir; }
    std::string fileName()  const { return m_fileName; }
    bool       append()     const { return m_append; }
    int        readCount()  const { return m_readCount; }
};

#endif
```

```C++
#ifndef APP_H
#define APP_H

#include "CLI.h"

class App {
public:
    int run(int argc, char** argv);
};

#endif
```